In [6]:
import argparse
import os
import numpy as np
import math
import sys
import torch
import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision import datasets

from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd


In [7]:
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU available: False
GPU name: No GPU


In [8]:
os.makedirs("images", exist_ok=True)

In [9]:
# Set parameters
n_epochs = 50  # Number of epochs of training
batch_size = 64  # Size of the batches
lr = 0.0002  # Adam: learning rate
b1 = 0.5  # Adam: decay of first order momentum of gradient
b2 = 0.999  # Adam: decay of second order momentum of gradient
latent_dim = 100  # Dimensionality of the latent space
img_size = 256  # Size of each image dimension (256x256)
channels = 3  # Number of image channels (RGB)
n_critic = 5  # Number of training steps for discriminator per iteration
clip_value = 0.01  # Lower and upper clip value for discriminator weights
sample_interval = 400  # Interval between image samples
lambda_gp = 10.0 # Define the gradient penalty weight

In [10]:
img_shape = (channels, img_size, img_size)
cuda = True if torch.cuda.is_available() else False

In [11]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img

In [12]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

In [15]:
dataset_dir = "dataset_train"  # Change this path accordingly

# Custom Dataset Class to load images
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith('.png')]  # Filter PNG files

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_name).convert('RGB')  # Open image and convert to RGB

        if self.transform:
            image = self.transform(image)

        return image



In [16]:
# Image transformation pipeline
transform = transforms.Compose([
    transforms.Resize(img_size),  # Resize to the target image size
    transforms.ToTensor(),  # Convert the image to tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize RGB images to [-1, 1]
])

# Create DataLoader
dataset = CustomImageDataset(root_dir=dataset_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Check the first batch of images
for batch in dataloader:
    print(batch.shape)  # Should output (batch_size, 3, 256, 256) for RGB images
    break


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/dataset_train'